In [2]:
from typing import List, Union
import deepdoctection as dd
from PyPDF2 import PdfFileWriter, PdfFileReader
from pathlib import Path
import os
import csv
import pandas as pd

In [2]:
def set_cfg():
    config_dict = {"CONFIG" : {"LAYOUT": "dd/d2/layout/CASCADE_RCNN_R_50_FPN_GN.yaml",
                               "TESS_OCR": "dd/conf_tesseract.yaml"
                               },
                   "WEIGHTS": {"LAYOUT" : "/home/janis/Experiments/fine_tune_xrf_d2/layout_190123/model_0001599.pth",
                               },
    "LANG": "deu",
    "WORD_MATCHING": {"PARENTAL_CATEGORIES": ["text", "title", "table","list", "figure"],
                      "RULE": "ioa",
                      "THRESHOLD": 0.6,
                      "MAX_PARENT_ONLY": True,
                      },
    "DEVICE": None,
                   }
    cfg = dd.AttrDict()
    cfg.from_dict(config_dict)
    return cfg


def build_analyzer(cfg: dd.AttrDict) -> dd.DoctectionPipe:

    pipe_component_list: List[dd.PipelineComponent] = []

    d_layout: Union[dd.D2FrcnnDetector, dd.TPFrcnnDetector]

    layout_config_path = dd.ModelCatalog.get_full_path_configs(cfg.CONFIG.LAYOUT)
    layout_weights_path = cfg.WEIGHTS.LAYOUT
    categories_layout = {
                "1": dd.LayoutType.text,
                "2": dd.LayoutType.title,
                "3": dd.LayoutType.list,
                "4": dd.LayoutType.table,
                "5": dd.LayoutType.figure,
            }
    d_layout = dd.D2FrcnnDetector(layout_config_path, layout_weights_path, categories_layout, device=cfg.DEVICE)
    
    layout = dd.ImageLayoutService(d_layout, to_image=True, crop_image=True)
    pipe_component_list.append(layout)


    tess_ocr_config_path = dd.get_configs_dir_path() / cfg.CONFIG.TESS_OCR
    d_tess_ocr = dd.TesseractOcrDetector(
    tess_ocr_config_path, config_overwrite=[f"LANGUAGES={cfg.LANG}"] if cfg.LANG is not None else None
    )
    text = dd.TextExtractionService(d_tess_ocr)
    pipe_component_list.append(text)

    match = dd.MatchingService(
        parent_categories=cfg.WORD_MATCHING.PARENTAL_CATEGORIES,
        child_categories=dd.LayoutType.word,
        matching_rule=cfg.WORD_MATCHING.RULE,
        threshold=cfg.WORD_MATCHING.THRESHOLD
        )
    pipe_component_list.append(match)

    order = dd.TextOrderService(
            text_container=dd.LayoutType.word,
            floating_text_block_names=[dd.LayoutType.title, dd.LayoutType.text, dd.LayoutType.list],
            text_block_names=[
                dd.LayoutType.title,
                dd.LayoutType.text,
                dd.LayoutType.list,
                dd.LayoutType.table, 
                dd.LayoutType.figure
            ],
    )
    pipe_component_list.append(order)

    pipe = dd.DoctectionPipe(pipeline_component_list=pipe_component_list)

    return pipe

/home/janis/Public/deepdoctection_pt/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0207 08:27.32 @file_utils.py:33]  INF  PyTorch version 1.9.0+cu111 available.


In [34]:
def reverse_pdf(path):
    output_pdf = PdfFileWriter()

    with open(path, 'rb') as readfile:
        path = Path(path)
        tmp_file_name = path.name.replace(".pdf","_tmp.pdf")
        input_pdf = PdfFileReader(readfile)
        total_pages = input_pdf.getNumPages()
        for page in range(total_pages - 1, -1, -1):
            output_pdf.addPage(input_pdf.getPage(page))
        tmp_path = (path.parent / tmp_file_name).as_posix()
        with open(tmp_path, "wb") as writefile:
            output_pdf.write(writefile)
    return tmp_path, total_pages

def find_references(path, dd_analyzer, total_pages):
    df = dd_analyzer.analyze(path=path)
    df.reset_state()
    is_looping = True
    for idx, dp in enumerate(df):
        for layout in dp.layouts:
            if layout.category_name == "title":
                if layout.text == "Literatur":
                    print(f"Reference found in page: {idx}")
                    is_looping = False  
                    break
        if not is_looping:
            break                        
    return total_pages - idx, idx + 1

def remove_unnecessary_pages(path, page_number, target_path):
    output_pdf = PdfFileWriter()
    file_name = Path(path).name.replace("_tmp.pdf",".pdf")
    target_file = Path(target_path) / file_name 
    with open(path, 'rb') as readfile:
        input_pdf = PdfFileReader(readfile)
        total_pages = input_pdf.getNumPages()
        for page in range(page_number):
            output_pdf.addPage(input_pdf.getPage(page))
        with open(target_file, "wb") as writefile:
            output_pdf.write(writefile)
    os.remove(path)
    
def read_csv():
    csv_path = "/home/janis/Data/filtered_docs.csv"


    with open(csv_path, 'r+') as f:
        reader = csv.reader(f, delimiter='\t')
        lines =[{"path": "/home/janis/Data/" + row[0].split(",")[0], "filtered": row[0].split(",")[1]} for idx, row in enumerate(reader) if idx>=3]
    return lines

def write_csv(lines):
    target_csv_path = "/home/janis/Data/filtered_docs_update.csv"
    df = pd.DataFrame(lines)
    df.to_csv(target_csv_path)
        
def launch():
    cfg = set_cfg()
    analyzer = build_analyzer(cfg)
    target_path = "/home/janis/Data/full_cut_files"
    
    lines = read_csv()
    lines = lines[1858:]
    for line in lines:
        if line["filtered"] == "1":
            print(f"Filtering: {line['path']}")
            line["processed"] = "1"
        else:
            print(f"Processing: {line['path']}")
            tmp_path, total_pages = reverse_pdf(line["path"])
            page_number_orig, page_number_tmp = find_references(tmp_path, analyzer, total_pages)
            remove_unnecessary_pages(tmp_path, page_number_tmp, target_path)
            line["page_number_orig"]=page_number_orig
            line["total_pages"]=total_pages
            line["page_number_cut_file"]=page_number_tmp
            line["processed"] = "1"
        write_csv(lines)

In [4]:
launch()

[0207 08:27.51 @config.py:66]  WRN  Loading config /home/janis/.cache/deepdoctection/configs/dd/d2/layout/CASCADE_RCNN_R_50_FPN_GN.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[0207 08:27.55 @checkpoint.py:150]  INF  [Checkpointer] Loading from /home/janis/Experiments/fine_tune_xrf_d2/layout_190123/model_0001599.pth ...
[0207 08:27.55 @c2_model_loading.py:324]  INF  Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                               | Shapes                          |
|:------------------------------------------------|:------------------------------------------------------------------|:--------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.weight,weight}    | (128,) (128,) (128,64,1,1)      |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.co

Processing: /home/janis/Data/full/c6640dd428e102a81a1524cb3fcd03d453ec5740.pdf


[0207 08:27.56 @doctectionpipe.py:118]  INF  Processing c6640dd428e102a81a1524cb3fcd03d453ec5740_tmp_0.pdf
/home/janis/Public/deepdoctection_pt/venv/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
[0207 08:27.56 @context.py:126]  INF  ImageLayoutService total: 0.1204 sec.
[0207 08:27.58 @context.py:126]  INF  TextExtractionService total: 2.1435 sec.
[0207 08:27.58 @context.py:126]  INF  MatchingService total: 0.0029 sec.
[0207 08:27.58 @context.py:126]  INF  TextOrderService total: 0.0183 sec.
[0207 08:27.59 @doctectionpipe.py:118]  INF  Processing c6640dd428e102a81a1524cb3fcd03d453ec5740_tmp_1.pdf
[0207 08:27.59 @context.py:126]  INF  ImageLayout

Reference found in page: 2
Processing: /home/janis/Data/full/c67a406bbe320c44c3c763e2016bc53115ca7a60.pdf


[0207 08:28.08 @doctectionpipe.py:118]  INF  Processing c67a406bbe320c44c3c763e2016bc53115ca7a60_tmp_0.pdf
[0207 08:28.08 @context.py:126]  INF  ImageLayoutService total: 0.0672 sec.
[0207 08:28.11 @context.py:126]  INF  TextExtractionService total: 2.7224 sec.
[0207 08:28.11 @context.py:126]  INF  MatchingService total: 0.0033 sec.
[0207 08:28.11 @context.py:126]  INF  TextOrderService total: 0.0227 sec.
[0207 08:28.12 @doctectionpipe.py:118]  INF  Processing c67a406bbe320c44c3c763e2016bc53115ca7a60_tmp_1.pdf
[0207 08:28.12 @context.py:126]  INF  ImageLayoutService total: 0.062 sec.
[0207 08:28.15 @context.py:126]  INF  TextExtractionService total: 2.7289 sec.
[0207 08:28.15 @context.py:126]  INF  MatchingService total: 0.0031 sec.
[0207 08:28.15 @context.py:126]  INF  TextOrderService total: 0.0206 sec.


...

[0207 10:10.41 @doctectionpipe.py:118]  INF  Processing ffa728a78965b756ba4989a76b642b27f6226529_tmp_0.pdf
[0207 10:10.41 @context.py:126]  INF  ImageLayoutService total: 0.0604 sec.
[0207 10:10.42 @context.py:126]  INF  TextExtractionService total: 1.3408 sec.
[0207 10:10.42 @context.py:126]  INF  MatchingService total: 0.0062 sec.
[0207 10:10.42 @context.py:126]  INF  TextOrderService total: 0.0307 sec.
[0207 10:10.43 @doctectionpipe.py:118]  INF  Processing ffa728a78965b756ba4989a76b642b27f6226529_tmp_1.pdf
[0207 10:10.43 @context.py:126]  INF  ImageLayoutService total: 0.0641 sec.
[0207 10:10.46 @context.py:126]  INF  TextExtractionService total: 2.7152 sec.
[0207 10:10.46 @context.py:126]  INF  MatchingService total: 0.004 sec.
[0207 10:10.46 @context.py:126]  INF  TextOrderService total: 0.0223 sec.


Reference found in page: 1
Processing: /home/janis/Data/full/ffbd0a1fa21ac40caa3074a39fc549d57d2d171d.pdf


[0207 10:10.47 @doctectionpipe.py:118]  INF  Processing ffbd0a1fa21ac40caa3074a39fc549d57d2d171d_tmp_0.pdf
[0207 10:10.47 @context.py:126]  INF  ImageLayoutService total: 0.0595 sec.
[0207 10:10.49 @context.py:126]  INF  TextExtractionService total: 1.7799 sec.
[0207 10:10.49 @context.py:126]  INF  MatchingService total: 0.0023 sec.
[0207 10:10.49 @context.py:126]  INF  TextOrderService total: 0.0119 sec.
[0207 10:10.50 @doctectionpipe.py:118]  INF  Processing ffbd0a1fa21ac40caa3074a39fc549d57d2d171d_tmp_1.pdf
[0207 10:10.50 @context.py:126]  INF  ImageLayoutService total: 0.0603 sec.
[0207 10:10.53 @context.py:126]  INF  TextExtractionService total: 2.6047 sec.
[0207 10:10.53 @context.py:126]  INF  MatchingService total: 0.0032 sec.
[0207 10:10.53 @context.py:126]  INF  TextOrderService total: 0.0193 sec.


Reference found in page: 1
Processing: /home/janis/Data/(Leer)


FileNotFoundError: [Errno 2] No such file or directory: '/home/janis/Data/(Leer)'

## Veredelung nach manueller Durchsicht

In [30]:
def read_csv():
    csv_path = "/home/janis/Data/filtered_docs_update.csv"


    with open(csv_path, 'r+') as f:
        reader = csv.reader(f, delimiter=',')
        base_path = Path("/home/janis/Data/full_cut_files")
        #lines =[{"path": "/home/janis/Data/" + row[0].split(",")[0], "filtered": row[0].split(",")[1]} for idx, row in enumerate(reader) if idx>=3]
        lines = [row for idx, row in enumerate(reader) if idx>=3]
        lines = [{"id": row[0], 
                  "path": str(base_path / Path(row[1]).name), 
                  "filtered_final": row[13], 
                  "veredelt": row[14], 
                  "page_number_cut_file": row[11]} for row in lines]
    return lines

def apply_final_filter(line):
    if int(line['filtered_final']):
        if os.path.isfile(line['path']):
            print(f"removing {line['path']}")
            os.remove(line['path'])

In [31]:
lines = read_csv()


In [32]:
for line in lines:
    apply_final_filter(line)

removing /home/janis/Data/full_cut_files/042366f146f178d76c377a191a414a876faf4b61.pdf
removing /home/janis/Data/full_cut_files/08d09f17d5d1074ad97b8626133404636c7ecf49.pdf
removing /home/janis/Data/full_cut_files/08fe8d0f5fe49ebda244108da4b8e8a81ce13ecf.pdf
removing /home/janis/Data/full_cut_files/0e08a5647a55b7350d400f794a11435fd56fc9c3.pdf
removing /home/janis/Data/full_cut_files/0eca191f1a5a114b7ef486444f4fdfb7f38d22f6.pdf
removing /home/janis/Data/full_cut_files/0ed0798d68ccda5357f0487509f930ec4b771cdd.pdf
removing /home/janis/Data/full_cut_files/1d5460e242a19760acb5ca293daf33466774012d.pdf
removing /home/janis/Data/full_cut_files/23c91620edaa3d0d88ba4c12cc7d5019dfef72da.pdf
removing /home/janis/Data/full_cut_files/24a26cbce160e3242b751e18e4cc7849d076d5bc.pdf
removing /home/janis/Data/full_cut_files/2ab2ce79cbedeff6f7ea141d21d080e5fb69a99b.pdf
removing /home/janis/Data/full_cut_files/2b8236ba9a0d57410f2bbfb41bcc47e4fd070486.pdf
removing /home/janis/Data/full_cut_files/2fa4198619cda

In [39]:
for line in lines:
    if int(line["veredelt"]) and not int(line["filtered_final"]):
        target_path = Path("/home/janis/Data/file_cut_files_tmp")
        remove_unnecessary_pages(line["path"],int(line['page_number_cut_file']),target_path)
        print(line["path"], line['page_number_cut_file'])

/home/janis/Data/full_cut_files/0ec69d5d86a250d6abf49be0ee02102f1cc433e0.pdf 3
/home/janis/Data/full_cut_files/1d47cefe96351f2a8d3c64bbd32c6bdef1aad8b9.pdf 3
/home/janis/Data/full_cut_files/1d529d232f1289cc3f350faa35d690864e1a4db5.pdf 5
/home/janis/Data/full_cut_files/1db9703641c62df5fae73f4f913857a8aa03bed8.pdf 3
/home/janis/Data/full_cut_files/218678f8b943c02b2d8510c6a9de02e0c8df8dad.pdf 2
/home/janis/Data/full_cut_files/275b00fae036d3bc8fde4730ee17df99caae57b7.pdf 2
/home/janis/Data/full_cut_files/28561aa0dc82ffd3114125edd89590105917f136.pdf 3
/home/janis/Data/full_cut_files/33688538efcbebf424452a7488be79859b4f0893.pdf 3
/home/janis/Data/full_cut_files/39495441710d98b2b73c5e9790cb119d8cc0b9c8.pdf 2
/home/janis/Data/full_cut_files/3a9ef084161bd9e5635273aa390dbe805b4823dc.pdf 1
/home/janis/Data/full_cut_files/41fc81a3bc27b896df812c5d3a4ecf740b78f59c.pdf 4
/home/janis/Data/full_cut_files/45a1f496c25bfc84db1147d023c8925bcaa26d5e.pdf 2
/home/janis/Data/full_cut_files/45aecd282a61bcb317b8

In [60]:
def remove_unnecessary_pages(path, page_number, target_path):
    output_pdf = PdfFileWriter()
    file_name = Path(path).name.replace("_tmp.pdf",".pdf")
    target_file = Path(target_path) / file_name 
    with open(path, 'rb') as readfile:
        input_pdf = PdfFileReader(readfile)
        total_pages = input_pdf.getNumPages()
        for page in range(page_number):
            output_pdf.addPage(input_pdf.getPage(page))
        with open(target_file, "wb") as writefile:
            output_pdf.write(writefile)
    os.remove(path)